In [1]:
from controller.game import GameController
from view.consolegui import ConsoleGui
from model.aiplayer import AiPlayer
from model.bag import Bag
from model.config import Direction
from model.row import Row
import numpy as np
from model.config import LETTER_VALUES, NO_CROSS_WORD, RACK_SIZE, BONUS

In [2]:
players = [None, None]
bag = Bag()
game = GameController(players, bag)
gui = ConsoleGui(game)


In [3]:
path = 'gcg/game_0001.gcg'

In [4]:
with open(path, 'r') as f:
    lines = f.readlines()

In [5]:
movelist = [line.split() for line in lines]
movelist

[['#player1', 'New_Player_1', 'New', 'Player', '1'],
 ['#player2', 'Quackle', 'Quackle'],
 ['>New_Player_1:', '?ABDGIR', '8D', 'BARDInG', '+76', '76'],
 ['>Quackle:', 'AEFLOSV', 'I3', 'FLAVO.ES', '+73', '73'],
 ['>New_Player_1:', 'AENRSWY', 'H1', 'WYE', '+32', '108'],
 ['>Quackle:', 'CEEHILR', '1A', 'CLERIHE.', '+101', '174'],
 ['>New_Player_1:', 'AABNPRS', '3G', 'B..ANAS', '+36', '144'],
 ['>Quackle:', 'AAEGIIK', '9C', 'KAIE', '+26', '200'],
 ['>New_Player_1:', 'ENPPRST', 'L2', 'P.RPENTS', '+76', '220'],
 ['>Quackle:', 'ADFGIIU', '10C', 'IGAD', '+33', '233'],
 ['>New_Player_1:', 'EEILNOR', 'B1', '.ONELIER', '+68', '288'],
 ['>Quackle:', 'FHIOUVW', 'A7', 'HOWF', '+50', '283'],
 ['>New_Player_1:', '?AINOTX', 'C6', 'OX', '+56', '344'],
 ['>Quackle:', 'CGITUUV', 'K5', 'UTU', '+18', '301'],
 ['>New_Player_1:', '?AEINTT', 'H8', '.NiTIATE', '+77', '421'],
 ['>Quackle:', 'CGIIMNV', '4A', 'M.VING', '+30', '331'],
 ['>New_Player_1:', 'ADEMNOQ', '14F', 'QA.', '+22', '443'],
 ['>Quackle:', 'CIJOS

In [6]:
player1 = AiPlayer(game, gui, movelist.pop(0)[1])
player2 = AiPlayer(game, gui, movelist.pop(0)[1])

In [7]:
game.players = [player1, player2]
game.active_player = player1 if player1.name == movelist[0][0][1:-1] else player2

In [8]:
current_move = movelist.pop(0)

In [9]:
current_move

['>New_Player_1:', '?ABDGIR', '8D', 'BARDInG', '+76', '76']

In [10]:
# we use '@' for blank for pragmatic reasons (it's the ASCII character before 'A')
# GCG uses '?', so let's fix that:
game.active_player.rack.rack_tiles = list(current_move[1].replace('?','@'))

In [11]:
game.active_player.rack

Rack object containing the following tiles:
@ABDGIR

In [12]:
# GCG has digit(s) followed by letter for horizontal move, or vice versa for vertical.
# We are expecting digit, then letter, then 'H' or 'V':
# while there are move lines left:

start_square = current_move[2]
if start_square[0].isdigit():
    start_square = start_square[-1] + start_square[:-1] + 'H'
else:
    start_square += 'V'
start_square

'D8H'

In [13]:
# lowercase letters in the GCG represent blanks. We're expecting a question mark for a blank, 
# followed by the desired letter, so replace 'a' with '?A', etc:

tiles = ''.join(['?'+letter.upper() if letter.islower() else letter for letter in current_move[3]])
tiles

'BARDI?NG'

In [14]:
# GCG gives the start square as the first letter in the word, 
# and uses '.' as a placeholder for any tile already on the board, 
# whereas we list the first square we're actually playing on, and
# the tiles actually played, so let's strip out '.' and adjust the
# starting square if necessary:

if '.' in tiles:
    start_square = list(start_square) # treat string as char list
    x = 0
    while tiles[x] == '.': # whilst there's a dot at the start
        if start_square[-1] == 'H':
            start_square[-2] = str(int(start_square[-2])+1) # increase the row if playing horiz.
        else:
            start_square[0] = chr(ord(start_square[0])+1) # or the column if vertical
        x += 1
    start_square = ''.join(start_square) # make a string again
    tiles = tiles.replace('.','') # strip out any dot in the middle of the word
    
                

In [15]:
game.active_player.rack

Rack object containing the following tiles:
@ABDGIR

In [16]:
start_square, tiles

('D8H', 'BARDI?NG')

In [17]:
move = gui.parse_move_string(start_square + tiles)

In [18]:
board = game.board
board

GameBoard object:
     A B C D E F G H I J K L M N O
   + - - - - - - - - - - - - - - - +
 1 | = . . ' . . . = . . . ' . . = |
 2 | . - . . . " . . . " . . . - . |
 3 | . . - . . . ' . ' . . . - . . |
 4 | ' . . - . . . ' . . . - . . ' |
 5 | . . . . - . . . . . - . . . . |
 6 | . " . . . " . . . " . . . " . |
 7 | . . ' . . . ' . ' . . . ' . . |
 8 | = . . ' . . . - . . . ' . . = |
 9 | . . ' . . . ' . ' . . . ' . . |
10 | . " . . . " . . . " . . . " . |
11 | . . . . - . . . . . - . . . . |
12 | . . . - . . . ' . . . - . . . |
13 | ' . - . . . ' . ' . . . - . ' |
14 | . - . . . " . . . " . . . - . |
15 | = . . ' . . . = . . . ' . . = |
   + - - - - - - - - - - - - - - - +
     A B C D E F G H I J K L M N O

In [19]:
player2.rack

Rack object containing the following tiles:
ADNNPRS

In [20]:
player1.rack.rack_tiles = move.tiles
player1.rack.reset_blanks()

In [21]:
moves = player1.generate_all_moves()

In [22]:
moves

[Move object:
 Move: G8, Horizontal, tiles: BA, score: 8.0, Move object:
 Move: E8, Horizontal, tiles: DeBAG, score: 16.0, Move object:
 Move: E8, Horizontal, tiles: DeBAR, score: 14.0, Move object:
 Move: E8, Horizontal, tiles: DeBAG, score: 16.0, Move object:
 Move: E8, Horizontal, tiles: DeBAR, score: 14.0, Move object:
 Move: E8, Horizontal, tiles: RIBA, score: 12.0, Move object:
 Move: G8, Horizontal, tiles: BAD, score: 12.0, Move object:
 Move: G8, Horizontal, tiles: BADe, score: 12.0, Move object:
 Move: G8, Horizontal, tiles: BADGe, score: 16.0, Move object:
 Move: G8, Horizontal, tiles: BADGeR, score: 20.0, Move object:
 Move: G8, Horizontal, tiles: BAG, score: 12.0, Move object:
 Move: E8, Horizontal, tiles: DeBAG, score: 16.0, Move object:
 Move: G8, Horizontal, tiles: BAGIe, score: 14.0, Move object:
 Move: G8, Horizontal, tiles: BAR, score: 10.0, Move object:
 Move: E8, Horizontal, tiles: DeBAR, score: 14.0, Move object:
 Move: G8, Horizontal, tiles: BARD, score: 14.0, Mov

In [23]:
game.execute_move(move)
game.change_active_player()
game.update_players()
board

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
game.validator.is_valid(move)
move.calculate_score()
move

In [ ]:
move.row

In [ ]:
game.board

In [ ]:
move.calculate_score()

In [ ]:
tile_values = [(LETTER_VALUES[(ord(t)-64)] if t.isupper() else LETTER_VALUES[0]) for t in move.tiles]
tile_values

In [ ]:
squares_in_whole_word = move.row.squares_in_word(move.start_index)
print(squares_in_whole_word)

In [ ]:
game.active_player.rack.rack_tiles

In [ ]:
board  = game.board
board

In [ ]:
import re

In [ ]:
re.sub(r'\?.', lambda m: m.group(0).lower()[-1], 'D8HBARD?ING')

In [ ]:
input_string = gui.strip_invalid_characters('D8HBARD?ING')

In [ ]:
input_string

In [ ]:
x = ord(input_string[0]) - 64
y = gui.digits_in_string(input_string)

In [ ]:
x, y

In [ ]:
input_string = gui.letters_in_string(input_string)

In [ ]:
input_string

In [ ]:
if input_string[1] == 'H':
    row = gui.board.get_row(y, Direction.HORIZONTAL)
    starting_square = x

In [ ]:
row

In [ ]:
input_string = input_string[2:]

In [ ]:
input_string = re.sub(r'\?.', lambda m: m.group(0).lower()[-1], input_string)

In [ ]:
input_string in gui.game.active_player.rack

In [ ]:
'BARDIG' in game.active_player.rack

In [ ]:
game.active_player.rack.rack_tiles = '@ABDGIR'

In [ ]:
str(game.active_player.rack)

In [ ]:
rack = game.active_player.rack

In [ ]:
'DGI' in rack